# Использование RePlay
Данный ноутбук показывает базовый порядок работы с библиотекой.
- загрузка датасета
- деление на трейн и тест
- обучение модели
- сравнение с бейзлайном

Документацию можно собрать из папки `docs` командой `make html`. Она создаст папку `_build` с документацией -- `/docs/_build/html/index.html`

In [1]:
%load_ext autoreload
%autoreload 2

Целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark.
Для отладки моделей (не на кластере) создаём локальную сессию.

Объект `State` позволяет использовать одну и ту же сессию Spark в разных объектах.
Если сессия уже инициализирована, то её нужно передать при инициализации `State`. Модули библиотеки, которым необходимо использовать спарк-сессию, например для конвертации из пандас в спарк, будут искать сессию именно в `State`.

По умолчанию создастся дефолтная сессия. Простой способ получить сессию с заданным количеством выделенной памяти -- функция `get_spark_session` модуля `session_handler`.

In [2]:
from replay.session_handler import State

spark = State().session
spark

In [3]:
K = 10

## 0. Подготовка данных <a name='data-preparator'></a>

В качестве примера возьмём датасет MovieLens 1m.

In [9]:
import pandas as pd
df = pd.read_csv("data/ml1m_ratings.dat", sep="\t", names=["user_id", "item_id", "relevance", "timestamp"])
users = pd.read_csv("data/ml1m_users.dat", sep="\t", names=["user_id", "gender", "age", "occupation", "zip_code"])

### 0.1. DataPreparator

Внутренний формат данных в библиотеке - spark dataframe.
Модели replay ожидают на вход pandas или spark dataframe, содержащий обязательные столбцы item_id, user_id. Для некоторых моделей обязательно наличие столбца relevance, содержащего оценки релевантности объектов для пользователей.
Для разбиения данных по времени с помощью Splitter и фильтров в логе должен присутсововать столбец timestamp.

В replay есть DataPreparator, который автоматически конвертирует pandas и spark dataframe c логом в нужный формат (переименование/создание обязательных столбцов, корректное чтение дат, проверка отсутствия пропусков). DataPreparetor может конвертировать dataframe с признаками пользователей/объектов (нужно указать один из столбцов, соответсвующий user_id/item_id.

Также конвертировать pandas dataframes в spark можно с помощью вспомогательной функции convert_to_spark.

In [10]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=df,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance",
        "timestamp": "timestamp"
    }
)

In [11]:
log.show(3)

+-------+-------+---------+-------------------+
|user_id|item_id|relevance|          timestamp|
+-------+-------+---------+-------------------+
|      1|   1193|      5.0|2001-01-01 01:12:40|
|      1|    661|      3.0|2001-01-01 01:35:09|
|      1|    914|      3.0|2001-01-01 01:32:48|
+-------+-------+---------+-------------------+
only showing top 3 rows



In [13]:
from replay.utils import convert2spark
items = convert2spark(users)

### 0.2. Split

Библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе.

`UserSplitter` отбирает для теста некоторое количество или долю объектов для каждого пользователя.

In [14]:
from replay.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234,
    shuffle=True
)
train, test = splitter.split(log)
(
    train.count(), 
    test.count()
)

(999709, 500)

## 1. Обучение модели 

#### SLIM
Один из простых, но эффективных алгоритмов 

In [15]:
from replay.models import SLIM

slim = SLIM(lambda_=0.01, beta=0.01)

/Users/darel/python/sponge-bob-magic/.new_env2/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [16]:
%%time

slim.fit(log=train)

02-Mar-21 17:21:32, replay, DEBUG: Начало обучения SLIM
DEBUG:replay:Начало обучения SLIM
02-Mar-21 17:21:32, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
02-Mar-21 17:21:33, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 2.49 s, sys: 161 ms, total: 2.65 s
Wall time: 6.86 s


In [17]:
%%time

recs = slim.predict(
    k=K,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

02-Mar-21 17:21:39, replay, DEBUG: Начало предикта SLIM
DEBUG:replay:Начало предикта SLIM


CPU times: user 964 ms, sys: 165 ms, total: 1.13 s
Wall time: 3.63 s


## 2. Оценка качества и сравнение результатов моделей

В библиотеке реализованы различные метрики качества рекомендательных систем, встречающихся в литературе.
Их можно использовать напрямую, либо запоминать результаты с помощью класса `Experiment`.

In [18]:
from replay.metrics import HitRate, NDCG, MAP
from replay.experiment import Experiment

metrics = Experiment(test, {NDCG(): K,
                            MAP() : K,
                            HitRate(): [1, int(K/2), K]})


In [19]:
%%time
metrics.add_result("SLIM", recs)
metrics.results

CPU times: user 138 ms, sys: 31.6 ms, total: 170 ms
Wall time: 1min 14s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.072,0.218,0.328,0.135137,0.18002


## 3. Конвертация в pandas

In [20]:
recs_pd = recs.toPandas()
recs_pd.head(3)

,user_id,item_id,relevance
0,1018,1200,0.843051
1,1018,1221,0.790657
2,1018,10,0.714412


## 4. Примеры использования других моделей RePlay

#### ALS
Библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию

In [21]:
from replay.models import ALSWrap

als = ALSWrap(rank=100)

In [22]:
%%time
als.fit(log=train)

02-Mar-21 17:22:58, replay, DEBUG: Начало обучения ALSWrap
DEBUG:replay:Начало обучения ALSWrap
02-Mar-21 17:22:58, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
02-Mar-21 17:22:59, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 312 ms, sys: 33.7 ms, total: 346 ms
Wall time: 37.2 s


In [23]:
%%time
recs = als.predict(
    k=K,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

02-Mar-21 17:23:36, replay, DEBUG: Начало предикта ALSWrap
DEBUG:replay:Начало предикта ALSWrap


CPU times: user 922 ms, sys: 154 ms, total: 1.08 s
Wall time: 3.15 s


In [24]:
%%time
metrics.add_result("ALS", recs)
metrics.results

CPU times: user 110 ms, sys: 23.9 ms, total: 133 ms
Wall time: 8.45 s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.072,0.218,0.328,0.135137,0.18002
ALS,0.064,0.258,0.376,0.149850,0.20300


#### MultVAE 
Пример использования DL в рекомендациях

In [25]:
from replay.models import MultVAE

multvae = MultVAE(epochs=100)

In [26]:
%%time
multvae.fit(log=train)

02-Mar-21 17:23:48, replay, DEBUG: Начало обучения MultVAE
DEBUG:replay:Начало обучения MultVAE
02-Mar-21 17:23:48, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
02-Mar-21 17:23:48, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
02-Mar-21 17:23:49, replay, DEBUG: Составление батча:
DEBUG:replay:Составление батча:
02-Mar-21 17:23:50, replay, DEBUG: Обучение модели
DEBUG:replay:Обучение модели
02-Mar-21 17:23:52, replay, DEBUG: Epoch[1] current loss: 1352.11820
DEBUG:replay:Epoch[1] current loss: 1352.11820
02-Mar-21 17:23:52, replay, DEBUG: Epoch[1] validation average loss: 1478.50049
DEBUG:replay:Epoch[1] validation average loss: 1478.50049
02-Mar-21 17:23:53, replay, DEBUG: Epoch[2] current loss: 1249.41315
DEBUG:replay:Epoch[2] current loss: 1249.41315
02-Mar-21 17:23:54, replay, DEBUG: Epoch[2] validation average loss: 1478.13831
DEBUG:replay:Epoch[2] validation average loss:

CPU times: user 1min 7s, sys: 6.19 s, total: 1min 13s
Wall time: 23.7 s


In [27]:
%%time

recs = multvae.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

02-Mar-21 17:24:11, replay, DEBUG: Начало предикта MultVAE
DEBUG:replay:Начало предикта MultVAE
02-Mar-21 17:24:14, replay, DEBUG: Предсказание модели
DEBUG:replay:Предсказание модели
/Users/darel/python/sponge-bob-magic/.new_env2/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


CPU times: user 1.03 s, sys: 186 ms, total: 1.21 s
Wall time: 8.83 s


In [28]:
%%time
metrics.add_result("MultVAE", recs)
metrics.results

CPU times: user 121 ms, sys: 29.1 ms, total: 150 ms
Wall time: 20 s


,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
SLIM,0.072,0.218,0.328,0.135137,0.180020
ALS,0.064,0.258,0.376,0.149850,0.203000
MultVAE,0.012,0.038,0.058,0.023180,0.031246


## 5 Сравнение результатов различных моделей
С помощью experiment можно сравнить качество моделей, построенных с использованием различных инструментов.
Предположим, мы хотим сравнить модели RePlay с некой "внешней" моделью. Для этого нужно:
* 5.1. Экспортировать обучающую выборку (в pandas/numpy/csv)
* 5.2. Обучить модель и получить рекомендации для всех пользователей в виде csv/pandas-датафрейма
* 5.3. Считать рекомендации с помощью DataPreparator
* 5.4. Посчитать метрики в experiment

#### 5.1 Экспортируем train

In [29]:
train.toPandas().to_csv("train.csv", index=False)

In [30]:
!head -n 5 train.csv

user_id,item_id,relevance,timestamp
1,1029,5.0,2001-01-01 01:36:45
1,2294,4.0,2001-01-07 02:38:11
1,3114,4.0,2001-01-01 01:36:14
1,783,4.0,2001-01-07 02:38:11


#### 5.2 Обучаем модель и получаем рекомендации в формате `id пользователя - id объекта - relevance`

Предположим, что это произошло, и у нас есть рекомендации в виде csv-файла. Ниже в качестве пример используем рекомендации, полученные одной из моделей, с рандомными релевантностями.

In [31]:
from pyspark.sql.functions import rand

In [32]:
recs.withColumn('relevance', rand(seed=123)).toPandas().to_csv("recs.csv", index=False)

#### 5.3 Теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой

In [33]:
recs = DataPreparator().transform(
    path="recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

#### 5.4 Сравним качество внешней модели с предыдущими результатами

In [34]:
metrics.add_result("my_model", recs)
metrics.results.sort_values("NDCG@10", ascending=False)

,HitRate@1,HitRate@5,HitRate@10,MAP@10,NDCG@10
ALS,0.064,0.258,0.376,0.149850,0.203000
SLIM,0.072,0.218,0.328,0.135137,0.180020
MultVAE,0.012,0.038,0.058,0.023180,0.031246
my_model,0.002,0.018,0.058,0.011301,0.021756
